In [1]:
import os, glob
import numpy as np
import pandas as pd
import math
import pickle
import matplotlib.pyplot as plt
#from scipy.stats import norm
# from scipy import stats
# from statsmodels.stats.weightstats import ztest
from IPython.display import display, HTML

In [2]:
non_coding_region="core_prom"
core_prom_type= "NonTATA"
cancer_type= "Lung"

In [3]:
base_dir = "/home/pdutta/Data/Cancer_wiseGDC/New_data/{}/Generated_files/DNABERT_Data/{}/Patient_wise".format(cancer_type, non_coding_region)
output_dir= "/home/pdutta/Data/Cancer_wiseGDC/New_data/{}/Generated_files/Variant_Analysis/{}/".format(cancer_type, core_prom_type)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [4]:
df_core_prom = pd.read_csv("/home/pdutta/Data/Noncoding_region/unique_core_prom_bed.tsv", sep="\t")
df_core_prom

,Chr,start,end,strand,TSS,gene_id,transcript_id,exon_id
0,chr1,11824,11913,+,11869,['ENSG00000223972.5'],['ENST00000456328.2'],['ENSE00002234944.1']
1,chr1,11965,12054,+,12010,['ENSG00000223972.5'],['ENST00000450305.2'],['ENSE00001948541.1']
2,chr1,29509,29598,+,29554,['ENSG00000243485.5'],['ENST00000473358.1'],['ENSE00001947070.1']
3,chr1,30222,30311,+,30267,['ENSG00000243485.5'],['ENST00000469289.1'],['ENSE00001841699.1']
4,chr1,30321,30410,+,30366,['ENSG00000284332.1'],['ENST00000607096.1'],['ENSE00003695741.1']
...,...,...,...,...,...,...,...,...
209272,chrY,57016052,57016141,-,57016096,['ENSG00000237801.6_PAR_Y'],['ENST00000412936.6_PAR_Y'],['ENSE00001527118.2']
209273,chrY,57165801,57165890,-,57165845,['ENSG00000228410.6_PAR_Y'],['ENST00000456370.6_PAR_Y'],['ENSE00001674980.1']
209274,chrY,57172725,57172814,-,57172769,['ENSG00000223484.7_PAR_Y'],['ENST00000421233.6_PAR_Y'],['ENSE00001611457.1']
209275,chrY,57203313,57203402,-,57203357,['ENSG00000185203.12_PAR_Y'],['ENST00000399966.9_PAR_Y'],['ENSE00001772131.3']


In [5]:
with open("/home/pdutta/Data/Cancer_wiseGDC/New_data/{}/Generated_files/Intersected_Data/intersected_vcf_{}_data.pkl".format(cancer_type, non_coding_region), "rb") as file:
    loaded_dictionary = pickle.load(file)

In [6]:
len(loaded_dictionary)

824

In [7]:
def log_oods_ratio(p1,p2):
    term1 = math.log2(p1/(1-p1))
    term2 = math.log2(p2/(1-p2))
    return (term1-term2)
def calculate_diff_probab(p1,p2):
    return((p2-p1)*max(p1,p2))

In [8]:
patient_dfs= []
patient_id = []
dir_remain = []  # List to keep track of directories where the file is missing

for dirpath, dirnames, filenames in os.walk(base_dir):
    #print(dirpath, dirnames, filenames)
    for dir_name in dirnames:
        if dir_name == '.ipynb_checkpoints':
            continue  # Skip the .ipynb_checkpoints folder
        
        dir_path = dirpath +"/" + dir_name
        print(dir_path)
        patient_id.append(dir_name)
        #print(dir_name)
        #input()
        df_temp = loaded_dictionary[dir_name]
        #print(len(df_temp))
        df_temp = df_temp.drop_duplicates(subset=['chr_name', 'start', 'end', 'transcript_id','START_POS', 'END_POS', 'REF','ALT']).reset_index()
        try:
            concat_probab = np.load(dir_path+"/{}/Prediction_result/pred_results.npy".format(core_prom_type))
            #print(len(concat_probab))
            #print(len(df_temp))
            if(2*len(df_temp)==len(concat_probab)):
                #input()
                Reference_probab = concat_probab[::2]
                Alternative_probab = concat_probab[1::2]
                df_temp['Ref_probab']=Reference_probab
                df_temp['Alt_probab']=Alternative_probab
                df_temp['ScoreChange'] = df_temp.apply(lambda row: calculate_diff_probab(row['Ref_probab'], row['Alt_probab']), axis=1)
                df_temp['LogOddRatio'] = df_temp.apply(lambda row: log_oods_ratio(row['Ref_probab'], row['Alt_probab']), axis=1)
                #display(df_temp)
                #input()
                # Apply the filter conditions
                df_temp = df_temp[((df_temp['Ref_probab'] > 0.5) & (df_temp['Alt_probab'] <= 0.5)) | 
                          ((df_temp['Ref_probab'] <= 0.5) & (df_temp['Alt_probab'] > 0.5))]
                #display(df_temp)
                #input()
                patient_dfs.append(df_temp)
            else:
                print(dir_name)
                input()
        except FileNotFoundError:
            print(f"File not found in directory: {dir_path}")
            dir_remain.append(dir_name)
        #input()
    break

/home/pdutta/Data/Cancer_wiseGDC/New_data/Lung/Generated_files/DNABERT_Data/core_prom/Patient_wise/c59b0462-855a-420e-a997-7c75f31c16ed_CaVEMan
/home/pdutta/Data/Cancer_wiseGDC/New_data/Lung/Generated_files/DNABERT_Data/core_prom/Patient_wise/7d0e7267-2303-428d-b42f-4b8cf1d69332_sanger_raw_pindel
/home/pdutta/Data/Cancer_wiseGDC/New_data/Lung/Generated_files/DNABERT_Data/core_prom/Patient_wise/c59b0462-855a-420e-a997-7c75f31c16ed_sanger_raw_pindel
/home/pdutta/Data/Cancer_wiseGDC/New_data/Lung/Generated_files/DNABERT_Data/core_prom/Patient_wise/514bbe99-1ce0-4c09-8931-d41b44758a86_CaVEMan
/home/pdutta/Data/Cancer_wiseGDC/New_data/Lung/Generated_files/DNABERT_Data/core_prom/Patient_wise/1766972a-0ea3-4230-9d94-db6bbad0a77a_CaVEMan
/home/pdutta/Data/Cancer_wiseGDC/New_data/Lung/Generated_files/DNABERT_Data/core_prom/Patient_wise/92113811-d6b8-436b-b491-ffe17ff2ae10_sanger_raw_pindel
/home/pdutta/Data/Cancer_wiseGDC/New_data/Lung/Generated_files/DNABERT_Data/core_prom/Patient_wise/bc2d1b6

In [9]:
dir_remain

[]

In [10]:
patient_dfs[0]

,index,chr_name,start,end,gene_id,transcript_id,exon_id,strand,TSS,CHROM,...,MP,SG,SNP,SP,TG,TP,Ref_probab,Alt_probab,ScoreChange,LogOddRatio
28,28,chr13,46466792,46466881,ENSG00000231817.9,ENST00000659419.1,ENSE00003879879.1,-,46466836,chr13,...,0.97,CC/CCTTT,False,0.200,CC/CCCTT,0.60,0.507973,0.455208,-0.026803,0.305197
32,32,chr13,46466777,46466866,ENSG00000231817.9,ENST00000661995.1,ENSE00003884887.1,-,46466821,chr13,...,0.97,CC/CCTTT,False,0.200,CC/CCCTT,0.60,0.600653,0.488423,-0.067412,0.655713
68,68,chr12,50764033,50764122,ENSG00000123268.9,ENST00000549612.5,ENSE00001374993.3,+,50764078,chr12,...,0.97,GG/CGGGG,False,0.430,GG/CCGGG,0.51,0.518703,0.450812,-0.035215,0.392758
73,73,chr1,152708115,152708204,ENSG00000187170.6,ENST00000368777.2,ENSE00001447943.1,+,152708160,chr1,...,0.92,TT/TTTTT,False,0.080,TT/ATTTT,0.88,0.351780,0.587309,0.138328,-1.390867
203,203,chr2,119370172,119370261,ENSG00000155368.17,ENST00000475783.1,ENSE00001895774.1,+,119370217,chr2,...,0.94,AA/AAATT,False,0.083,AA/AAAAT,0.86,0.392538,0.530220,0.073002,-0.804569


In [11]:
# Concatenate the patient dataframes
concatenated_df = pd.concat(patient_dfs)

# Group by specified key columns
grouped_df = concatenated_df.groupby(['chr_name', 'start', 'end', 'START_POS', 'END_POS', 'REF', 'ALT'])

# Aggregate various columns differently
unique_patient_regions = grouped_df.agg({
    'transcript_id': lambda x: ', '.join(x.unique()),  # Comma-separated list of unique values
    'gene_id': lambda x: ', '.join(x.unique()),  # Same for gene_id
    'exon_id': lambda x: ', '.join(x.unique()),  # And for exon_id
    'strand': 'first',  # Assuming it does not vary
    'TSS': 'first',  # Assuming it does not vary
    'CHROM': 'first',  # Assuming it does not vary
    'ID': lambda x: ', '.join(x.unique()),  # If ID can vary, aggregate uniquely
    'QUAL': 'first',  # Average quality score might be meaningful
    'FILTER': 'first',  # Generally does not vary if other key fields don't
    # For boolean or categorical data, use the first or mode
    'SNP': 'first',  # Assuming SNP designation does not vary
    'DP': 'sum',  # Total read depth might be aggregated by sum
    'Ref_probab': 'first',  # Mean probabilities for reference alleles
    'Alt_probab': 'first',  # Mean probabilities for alternate alleles
    'ScoreChange': 'first',  # Sum of score changes might be useful
    'LogOddRatio': 'first',  # Average of log-odds ratios
    # Continue for other fields as appropriate
}).reset_index()

# Display the modified DataFrame structure
unique_patient_regions

,chr_name,start,end,START_POS,END_POS,REF,ALT,transcript_id,gene_id,exon_id,...,CHROM,ID,QUAL,FILTER,SNP,DP,Ref_probab,Alt_probab,ScoreChange,LogOddRatio
0,chr1,778702,778791,778705,778706,A,G,"ENST00000412115.2, ENST00000655765.1",ENSG00000237491.10,ENSE00003853005.1,...,chr1,961ec902-994f-11ea-b474-9b068d153a83,.,VUM,False,66.0,0.423698,0.746212,0.240664,-1.999754
1,chr1,827553,827642,827586,827592,GGGGCC,G,ENST00000445118.7,ENSG00000228794.10,ENSE00001768150.2,...,chr1,"7153e194-970f-11ea-b475-f7eb269f556d, f7f754b8...",120.0,FF010,None,0.0,0.808358,0.426837,-0.308405,2.501840
2,chr1,1013452,1013541,1013465,1013466,T,A,ENST00000649529.1,ENSG00000187608.10,ENSE00003831768.1,...,chr1,"e6736ef2-98ab-11ea-8820-e99dcf289f81, 5b327d22...",.,VUM,False,440.0,0.439124,0.525944,0.045662,-0.502902
3,chr1,1013452,1013541,1013467,1013468,T,A,ENST00000649529.1,ENSG00000187608.10,ENSE00003831768.1,...,chr1,43c063b4-fedc-11ed-9a06-818a8d593b68,.,GI,False,108.0,0.439124,0.699538,0.182169,-1.572269
4,chr1,1116317,1116406,1116393,1116394,G,T,ENST00000475119.5,ENSG00000131591.18,ENSE00001917779.1,...,chr1,016f7df4-9820-11ea-99cd-73bdaa383a70,.,PASS,False,66.0,0.546520,0.491788,-0.029912,0.316624
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4779,chrY,11314876,11314965,11314888,11314889,C,T,ENST00000557360.1,ENSG00000259154.1,ENSE00002516975.1,...,chrY,"9787e9e8-fc68-11ed-b310-7a175a442458, df513dea...",.,MN,False,4328.0,0.717151,0.327176,-0.279671,2.382406
4780,chrY,11314876,11314965,11314896,11314897,A,T,ENST00000557360.1,ENSG00000259154.1,ENSE00002516975.1,...,chrY,"229ff4a4-9ea6-11ea-bf95-ea9c548dde32, 9787ea60...",.,SE,False,1604.0,0.717151,0.412062,-0.218795,1.855046
4781,chrY,11314876,11314965,11314935,11314936,C,A,ENST00000557360.1,ENSG00000259154.1,ENSE00002516975.1,...,chrY,"cb26c3ac-9a26-11ea-b54d-c4c9f27ff9af, 9f54bde8...",.,MN,False,2343.0,0.717150,0.438935,-0.199522,1.696399
4782,chrY,11314876,11314965,11314945,11314946,G,A,ENST00000557360.1,ENSG00000259154.1,ENSE00002516975.1,...,chrY,3c47153a-fd27-11ed-a29f-001d390521d5,.,MN,False,55.0,0.717151,0.462559,-0.182581,1.558715


In [12]:
filtered_regions = unique_patient_regions.groupby(['chr_name', 'transcript_id', 'start', 'end']).filter(lambda x: len(x) > 1 and not all(x['START_POS'].eq(x['START_POS'].iloc[0]) & x['END_POS'].eq(x['END_POS'].iloc[0]) & x['REF'].eq(x['REF'].iloc[0]) & x['ALT'].eq(x['ALT'].iloc[0])))
filtered_regions

,chr_name,start,end,START_POS,END_POS,REF,ALT,transcript_id,gene_id,exon_id,...,CHROM,ID,QUAL,FILTER,SNP,DP,Ref_probab,Alt_probab,ScoreChange,LogOddRatio
2,chr1,1013452,1013541,1013465,1013466,T,A,ENST00000649529.1,ENSG00000187608.10,ENSE00003831768.1,...,chr1,"e6736ef2-98ab-11ea-8820-e99dcf289f81, 5b327d22...",.,VUM,False,440.0,0.439124,0.525944,0.045662,-0.502902
3,chr1,1013452,1013541,1013467,1013468,T,A,ENST00000649529.1,ENSG00000187608.10,ENSE00003831768.1,...,chr1,43c063b4-fedc-11ed-9a06-818a8d593b68,.,GI,False,108.0,0.439124,0.699538,0.182169,-1.572269
9,chr1,1583865,1583954,1583936,1583938,TA,T,ENST00000621860.1,ENSG00000274481.1,ENSE00003719557.1,...,chr1,"d7d854e8-9f7c-11ea-bc84-c14a6e61a702, 322dc916...",534.0,FF010,None,0.0,0.517109,0.402054,-0.059496,0.671395
10,chr1,1583865,1583954,1583941,1583942,A,T,ENST00000621860.1,ENSG00000274481.1,ENSE00003719557.1,...,chr1,0a0ca8ae-a00b-11ea-a8bc-f7a36e61a702,.,MNP,False,51.0,0.517109,0.402054,-0.059496,0.671395
13,chr1,3900307,3900396,3900311,3900312,A,C,ENST00000667408.1,ENSG00000236423.6,ENSE00003884328.1,...,chr1,123df6e2-24cf-11eb-a6a0-463b27697200,.,MNP,False,44.0,0.393931,0.704957,0.219260,-1.878151
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4778,chrY,11314876,11314965,11314883,11314884,G,A,ENST00000557360.1,ENSG00000259154.1,ENSE00002516975.1,...,chrY,"d3fd3afa-1d76-11eb-b58b-194bbcf5dd3d, 3c4714b8...",.,MQ,False,1552.0,0.717151,0.453449,-0.189114,1.611663
4779,chrY,11314876,11314965,11314888,11314889,C,T,ENST00000557360.1,ENSG00000259154.1,ENSE00002516975.1,...,chrY,"9787e9e8-fc68-11ed-b310-7a175a442458, df513dea...",.,MN,False,4328.0,0.717151,0.327176,-0.279671,2.382406
4780,chrY,11314876,11314965,11314896,11314897,A,T,ENST00000557360.1,ENSG00000259154.1,ENSE00002516975.1,...,chrY,"229ff4a4-9ea6-11ea-bf95-ea9c548dde32, 9787ea60...",.,SE,False,1604.0,0.717151,0.412062,-0.218795,1.855046
4781,chrY,11314876,11314965,11314935,11314936,C,A,ENST00000557360.1,ENSG00000259154.1,ENSE00002516975.1,...,chrY,"cb26c3ac-9a26-11ea-b54d-c4c9f27ff9af, 9f54bde8...",.,MN,False,2343.0,0.717150,0.438935,-0.199522,1.696399


In [13]:
pd.set_option('display.max_columns', None)
filtered_regions[filtered_regions['start']==47977248]

,chr_name,start,end,START_POS,END_POS,REF,ALT,transcript_id,gene_id,exon_id,strand,TSS,CHROM,ID,QUAL,FILTER,SNP,DP,Ref_probab,Alt_probab,ScoreChange,LogOddRatio
4315,chr8,47977248,47977337,47977261,47977263,TC,T,ENST00000648533.1,ENSG00000104738.18,ENSE00003840279.1,+,47977293,chr8,"bd4c8f2e-1b8a-11eb-baba-694fc5164950, 774b800e...",240.0,FF010,None,0.0,0.961837,0.368681,-0.570519,5.431526
4316,chr8,47977248,47977337,47977261,47977264,TCA,T,ENST00000648533.1,ENSG00000104738.18,ENSE00003840279.1,+,47977293,chr8,"67dafdf6-9abf-11ea-9300-cbc745bdced2, cb469944...",180.0,FF010,None,0.0,0.961837,0.274365,-0.661236,6.058677
4317,chr8,47977248,47977337,47977262,47977264,CA,C,ENST00000648533.1,ENSG00000104738.18,ENSE00003840279.1,+,47977293,chr8,"95bf348e-9d1d-11ea-ada6-b34d31df1b47, 7d7ad15a...",420.0,FF006,None,0.0,0.961837,0.302388,-0.634282,5.861552
4318,chr8,47977248,47977337,47977262,47977265,CAA,C,ENST00000648533.1,ENSG00000104738.18,ENSE00003840279.1,+,47977293,chr8,"d0b21748-9b4f-11ea-ad28-147dc0e32e4c, efc61d58...",600.0,FF006,None,0.0,0.961837,0.287415,-0.648683,5.965453
4319,chr8,47977248,47977337,47977284,47977286,AC,A,ENST00000648533.1,ENSG00000104738.18,ENSE00003840279.1,+,47977293,chr8,"b31f3872-f6f6-11ed-9735-b9bc2b5aff55, 93d0bd8a...",180.0,FF004,None,0.0,0.961837,0.265422,-0.669837,6.124154


In [14]:
# Reset the option to its default (typically 20)
pd.reset_option('display.max_columns')

In [15]:
unique_patient_regions[unique_patient_regions['start']==47977248]

,chr_name,start,end,START_POS,END_POS,REF,ALT,transcript_id,gene_id,exon_id,...,CHROM,ID,QUAL,FILTER,SNP,DP,Ref_probab,Alt_probab,ScoreChange,LogOddRatio
4315,chr8,47977248,47977337,47977261,47977263,TC,T,ENST00000648533.1,ENSG00000104738.18,ENSE00003840279.1,...,chr8,"bd4c8f2e-1b8a-11eb-baba-694fc5164950, 774b800e...",240.0,FF010,None,0.0,0.961837,0.368681,-0.570519,5.431526
4316,chr8,47977248,47977337,47977261,47977264,TCA,T,ENST00000648533.1,ENSG00000104738.18,ENSE00003840279.1,...,chr8,"67dafdf6-9abf-11ea-9300-cbc745bdced2, cb469944...",180.0,FF010,None,0.0,0.961837,0.274365,-0.661236,6.058677
4317,chr8,47977248,47977337,47977262,47977264,CA,C,ENST00000648533.1,ENSG00000104738.18,ENSE00003840279.1,...,chr8,"95bf348e-9d1d-11ea-ada6-b34d31df1b47, 7d7ad15a...",420.0,FF006,None,0.0,0.961837,0.302388,-0.634282,5.861552
4318,chr8,47977248,47977337,47977262,47977265,CAA,C,ENST00000648533.1,ENSG00000104738.18,ENSE00003840279.1,...,chr8,"d0b21748-9b4f-11ea-ad28-147dc0e32e4c, efc61d58...",600.0,FF006,None,0.0,0.961837,0.287415,-0.648683,5.965453
4319,chr8,47977248,47977337,47977284,47977286,AC,A,ENST00000648533.1,ENSG00000104738.18,ENSE00003840279.1,...,chr8,"b31f3872-f6f6-11ed-9735-b9bc2b5aff55, 93d0bd8a...",180.0,FF004,None,0.0,0.961837,0.265422,-0.669837,6.124154


In [16]:
# Step 3: Create a new dataframe with three rows for each patient, with columns as the acceptor coordinates
# Initialize the dictionaries
data_ref, data_alt, data_log_odd, data_score_change = {}, {}, {}, {}

# Loop over each region in the unique_patient_regions
for _, region in unique_patient_regions.iterrows():
    key = f'{region.chr_name}_{region.start}_{region.end}_{region.START_POS}_{region.END_POS}_{region.REF}_{region.ALT}'
    data_ref[key] = []
    data_alt[key] = []
    data_log_odd[key] = []
    data_score_change[key] = []

# Loop through each patient and for each region, get the Alt_probab score for that region for that patient
for df in patient_dfs:
    print(df.shape)
    #print(df)
    for _, region in unique_patient_regions.iterrows():
        #print(region)
        #input()
        for data, score_column in zip([data_ref, data_alt, data_log_odd, data_score_change], ['Ref_probab', 'Alt_probab', 'LogOddRatio', 'ScoreChange']):
            score_series = df.loc[(df['chr_name'] == region.chr_name) & (df['start'] == region.start)& (df['end'] == region.end) & (df['START_POS'] == region.START_POS) & (df['END_POS'] == region.END_POS) & (df['REF'] == region.REF) & (df['ALT'] == region.ALT) , score_column]
            #print(score_series)
            if score_series.empty:
                score = np.nan
            else:
                score = score_series.max()
                #print(score_series , score)
                #input()
            data[f'{region.chr_name}_{region.start}_{region.end}_{region.START_POS}_{region.END_POS}_{region.REF}_{region.ALT}'].append(score)

(5, 33)
(105, 30)
(80, 30)
(56, 33)
(18, 33)
(89, 30)
(37, 33)
(91, 30)
(18, 33)
(83, 30)
(103, 30)
(15, 33)
(11, 33)
(68, 33)
(102, 30)
(11, 33)
(69, 30)
(17, 33)
(68, 30)
(64, 30)
(11, 33)
(81, 30)
(101, 30)
(5, 33)
(72, 30)
(11, 33)
(9, 33)
(20, 33)
(85, 30)
(84, 30)
(6, 33)
(84, 30)
(105, 30)
(73, 30)
(83, 33)
(71, 33)
(90, 30)
(102, 30)
(110, 30)
(104, 30)
(106, 30)
(18, 33)
(8, 33)
(122, 30)
(77, 30)
(88, 30)
(86, 30)
(87, 30)
(9, 33)
(65, 33)
(9, 33)
(73, 30)
(10, 33)
(89, 30)
(8, 33)
(119, 30)
(71, 30)
(10, 33)
(114, 30)
(95, 30)
(60, 33)
(21, 33)
(7, 33)
(89, 30)
(48, 33)
(6, 33)
(83, 33)
(89, 30)
(23, 33)
(53, 33)
(95, 30)
(15, 33)
(3, 33)
(110, 30)
(10, 33)
(70, 30)
(113, 30)
(38, 33)
(76, 30)
(15, 33)
(89, 30)
(93, 30)
(82, 30)
(10, 33)
(86, 30)
(100, 30)
(92, 30)
(85, 30)
(107, 30)
(78, 30)
(20, 33)
(6, 33)
(113, 30)
(8, 33)
(111, 30)
(23, 33)
(72, 30)
(21, 33)
(11, 33)
(9, 33)
(72, 33)
(15, 33)
(85, 30)
(89, 30)
(93, 30)
(9, 33)
(106, 30)
(81, 30)
(87, 30)
(5, 33)
(78, 30

In [17]:
result_df_ref = pd.DataFrame(data_ref)
result_df_ref.insert(0, "Patient_ID", patient_id)
result_df_ref

,Patient_ID,chr1_778702_778791_778705_778706_A_G,chr1_827553_827642_827586_827592_GGGGCC_G,chr1_1013452_1013541_1013465_1013466_T_A,chr1_1013452_1013541_1013467_1013468_T_A,chr1_1116317_1116406_1116393_1116394_G_T,chr1_1390856_1390945_1390932_1390933_C_A,chr1_1398322_1398411_1398332_1398333_T_C,chr1_1399485_1399574_1399492_1399493_G_A,chr1_1503205_1503294_1503222_1503223_C_CA,...,chrY_10092949_10093038_10092950_10092951_G_T,chrY_10092949_10093038_10092980_10092981_G_T,chrY_11306873_11306962_11306954_11306955_C_T,chrY_11314876_11314965_11314879_11314880_G_A,chrY_11314876_11314965_11314883_11314884_G_A,chrY_11314876_11314965_11314888_11314889_C_T,chrY_11314876_11314965_11314896_11314897_A_T,chrY_11314876_11314965_11314935_11314936_C_A,chrY_11314876_11314965_11314945_11314946_G_A,chrY_25174475_25174564_25174484_25174485_T_C
0,c59b0462-855a-420e-a997-7c75f31c16ed_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7d0e7267-2303-428d-b42f-4b8cf1d69332_sanger_ra...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,c59b0462-855a-420e-a997-7c75f31c16ed_sanger_ra...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,514bbe99-1ce0-4c09-8931-d41b44758a86_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1766972a-0ea3-4230-9d94-db6bbad0a77a_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
819,fabc8ac4-f575-40c3-9c29-f4a633b0fdd9_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
820,1d6d39c4-b0e1-4d49-ac1d-384212229d5c_sanger_ra...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
821,099b611a-4414-44c3-9458-89b01425d7a4_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
822,14428874-2d28-48a5-8a7e-eef30da45e47_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
result_df_alt = pd.DataFrame(data_alt)
result_df_alt.insert(0, "Patient_ID", patient_id)
result_df_alt

,Patient_ID,chr1_778702_778791_778705_778706_A_G,chr1_827553_827642_827586_827592_GGGGCC_G,chr1_1013452_1013541_1013465_1013466_T_A,chr1_1013452_1013541_1013467_1013468_T_A,chr1_1116317_1116406_1116393_1116394_G_T,chr1_1390856_1390945_1390932_1390933_C_A,chr1_1398322_1398411_1398332_1398333_T_C,chr1_1399485_1399574_1399492_1399493_G_A,chr1_1503205_1503294_1503222_1503223_C_CA,...,chrY_10092949_10093038_10092950_10092951_G_T,chrY_10092949_10093038_10092980_10092981_G_T,chrY_11306873_11306962_11306954_11306955_C_T,chrY_11314876_11314965_11314879_11314880_G_A,chrY_11314876_11314965_11314883_11314884_G_A,chrY_11314876_11314965_11314888_11314889_C_T,chrY_11314876_11314965_11314896_11314897_A_T,chrY_11314876_11314965_11314935_11314936_C_A,chrY_11314876_11314965_11314945_11314946_G_A,chrY_25174475_25174564_25174484_25174485_T_C
0,c59b0462-855a-420e-a997-7c75f31c16ed_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7d0e7267-2303-428d-b42f-4b8cf1d69332_sanger_ra...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,c59b0462-855a-420e-a997-7c75f31c16ed_sanger_ra...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,514bbe99-1ce0-4c09-8931-d41b44758a86_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1766972a-0ea3-4230-9d94-db6bbad0a77a_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
819,fabc8ac4-f575-40c3-9c29-f4a633b0fdd9_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
820,1d6d39c4-b0e1-4d49-ac1d-384212229d5c_sanger_ra...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
821,099b611a-4414-44c3-9458-89b01425d7a4_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
822,14428874-2d28-48a5-8a7e-eef30da45e47_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
result_df_score_change = pd.DataFrame(data_score_change)
result_df_score_change.insert(0, "Patient_ID", patient_id)
result_df_score_change

,Patient_ID,chr1_778702_778791_778705_778706_A_G,chr1_827553_827642_827586_827592_GGGGCC_G,chr1_1013452_1013541_1013465_1013466_T_A,chr1_1013452_1013541_1013467_1013468_T_A,chr1_1116317_1116406_1116393_1116394_G_T,chr1_1390856_1390945_1390932_1390933_C_A,chr1_1398322_1398411_1398332_1398333_T_C,chr1_1399485_1399574_1399492_1399493_G_A,chr1_1503205_1503294_1503222_1503223_C_CA,...,chrY_10092949_10093038_10092950_10092951_G_T,chrY_10092949_10093038_10092980_10092981_G_T,chrY_11306873_11306962_11306954_11306955_C_T,chrY_11314876_11314965_11314879_11314880_G_A,chrY_11314876_11314965_11314883_11314884_G_A,chrY_11314876_11314965_11314888_11314889_C_T,chrY_11314876_11314965_11314896_11314897_A_T,chrY_11314876_11314965_11314935_11314936_C_A,chrY_11314876_11314965_11314945_11314946_G_A,chrY_25174475_25174564_25174484_25174485_T_C
0,c59b0462-855a-420e-a997-7c75f31c16ed_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7d0e7267-2303-428d-b42f-4b8cf1d69332_sanger_ra...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,c59b0462-855a-420e-a997-7c75f31c16ed_sanger_ra...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,514bbe99-1ce0-4c09-8931-d41b44758a86_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1766972a-0ea3-4230-9d94-db6bbad0a77a_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
819,fabc8ac4-f575-40c3-9c29-f4a633b0fdd9_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
820,1d6d39c4-b0e1-4d49-ac1d-384212229d5c_sanger_ra...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
821,099b611a-4414-44c3-9458-89b01425d7a4_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
822,14428874-2d28-48a5-8a7e-eef30da45e47_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
result_df_log_odd = pd.DataFrame(data_log_odd)
result_df_log_odd.insert(0, "Patient_ID", patient_id)
result_df_log_odd

,Patient_ID,chr1_778702_778791_778705_778706_A_G,chr1_827553_827642_827586_827592_GGGGCC_G,chr1_1013452_1013541_1013465_1013466_T_A,chr1_1013452_1013541_1013467_1013468_T_A,chr1_1116317_1116406_1116393_1116394_G_T,chr1_1390856_1390945_1390932_1390933_C_A,chr1_1398322_1398411_1398332_1398333_T_C,chr1_1399485_1399574_1399492_1399493_G_A,chr1_1503205_1503294_1503222_1503223_C_CA,...,chrY_10092949_10093038_10092950_10092951_G_T,chrY_10092949_10093038_10092980_10092981_G_T,chrY_11306873_11306962_11306954_11306955_C_T,chrY_11314876_11314965_11314879_11314880_G_A,chrY_11314876_11314965_11314883_11314884_G_A,chrY_11314876_11314965_11314888_11314889_C_T,chrY_11314876_11314965_11314896_11314897_A_T,chrY_11314876_11314965_11314935_11314936_C_A,chrY_11314876_11314965_11314945_11314946_G_A,chrY_25174475_25174564_25174484_25174485_T_C
0,c59b0462-855a-420e-a997-7c75f31c16ed_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7d0e7267-2303-428d-b42f-4b8cf1d69332_sanger_ra...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,c59b0462-855a-420e-a997-7c75f31c16ed_sanger_ra...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,514bbe99-1ce0-4c09-8931-d41b44758a86_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1766972a-0ea3-4230-9d94-db6bbad0a77a_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
819,fabc8ac4-f575-40c3-9c29-f4a633b0fdd9_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
820,1d6d39c4-b0e1-4d49-ac1d-384212229d5c_sanger_ra...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
821,099b611a-4414-44c3-9458-89b01425d7a4_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
822,14428874-2d28-48a5-8a7e-eef30da45e47_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
result_df_ref.to_csv(output_dir+"df_ref_score.tsv", sep="\t", index=False)
result_df_alt.to_csv(output_dir+"df_alt_score.tsv", sep="\t", index=False)
result_df_log_odd.to_csv(output_dir+"df_log_odd_score.tsv", sep="\t", index=False)
result_df_score_change.to_csv(output_dir+"df_score_change.tsv", sep="\t", index=False)